In [ ]:
import sys
sys.path.insert(0, './src')  # 或者绝对路径：/full/path/to/your_project
from src.xmcdata import *
from gen_model import *

In [ ]:
# Example configurations for different models
data_dir = "./xmc-base/"
dataset_name = "eurlex-4k"
dataset_dir = data_dir+dataset_name
model_names = [
    "google/flan-t5-base",  # T5-base
    "google/flan-t5-large",  # T5-large
    "google/flan-t5-xl",     # T5-xl
    "facebook/bart-base",    # BART
    "google/pegasus-xsum"    # Pegasus
]
# T5-FLAN Configuration
t5_config = T2TConfig(
    model_name=model_names[0],  # Change to other models as needed
    dataset_dir=dataset_dir,
    output_dir=data_dir+"outputs/"+model_names[0].split("/")[-1],
    prompt="Summarize this document by keyphrases:",
    #task_prefix="classify:",  # T5-style task prefix
    num_epochs=3,
    batch_size=4,
    max_input_length=512,
    max_output_length=128,
    strict_tokenization=True
)

# BART Configuration
bart_config = T2TConfig(
    model_name=model_names[3],  # Change to other models as needed
    dataset_dir=dataset_dir,
    output_dir=data_dir+"outputs/"+model_names[3].split("/")[-1],
    prompt="Summarize this document by keyphrases:",
    num_epochs=3,
    batch_size=4,
    max_input_length=512,
    max_output_length=128,
    strict_tokenization=True  # Apply T5-style strict standards to BART
)

# Pegasus Configuration
pegasus_config = T2TConfig(
    model_name=model_names[4],  # Change to other models as needed
    dataset_dir=dataset_dir,
    output_dir=data_dir+"outputs/"+model_names[4].split("/")[-1],
    prompt="Summarize this document by keyphrases:",
    num_epochs=3,
    batch_size=4,  # Smaller batch for Pegasus
    max_input_length=512,
    max_output_length=128,
    strict_tokenization=True  # Apply T5-style strict standards to Pegasus
)

In [ ]:
# Choose configuration
config = t5_config  # Change to bart_config or pegasus_config as needed

# Initialize model
model = UniversalT2TModel(config)

# Save configuration
model.save_config()

# Train the model (uncomment to train)
model.train()

In [ ]:
# Generate predictions for test set
print("Generating predictions with T5-style strict processing...")
predictions = model.predict_test_set(output_path=dataset_dir)

print(f"Generated {len(predictions)} predictions using {model.tokenizer.model_type.upper()}")
print("Sample predictions:")
for i, pred in enumerate(predictions[:3]):
    print(f"{i+1}: {pred}")